<a href="https://colab.research.google.com/github/abhishekrapid/learning-langchain-ai/blob/PYT-106-r-d-on-lang-chain/QnA%20from%20the%20PDF%20file%20using%20OpenAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install the required packages
# This is not necessary if the packages have already been installed
# using pip or a package manager like conda
!pip install langchain
!pip install unstructured
!pip install unstructured[local-inference]
!pip install pinecone-client
!pip install tiktoken
!pip install openai
!pip install python-dotenv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 600.7/600.7 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 41.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 55.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 7.3 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.9
    Uninstalling SQLAlchemy-2.0.9:
      Successfully uninstalled SQLAlchemy-2.0.9
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.2/177.2 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 24.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 24.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 3.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Import the required packages
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma, Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain
import pinecone
from dotenv import load_dotenv
import os

load_dotenv()

/usr/local/lib/python3.9/dist-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


True

In [ ]:
# Load the required modules from the packages
# The modules will be used later in the code
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_API_ENV = os.getenv("PINECONE_API_ENV")
PINECONE_SERVICE_NAME = os.getenv("PINECONE_SERVICE_NAME")

In [ ]:
# Define a function to load a document from a URL
def load_document(url):
    if url.startswith("http"):
        loader = OnlinePDFLoader(url)
    else:
        loader = UnstructuredPDFLoader(url)

    return loader.load()

In [ ]:
online_pdf_loader = load_document("https://wolfpaulus.com/wp-content/uploads/2017/05/field-guide-to-data-science.pdf")
print(online_pdf_loader)

[Document(page_content='DATA SCIENCEtoTHEFIELD GUIDE     SECOND EDITION© COPYRIGHT 2015 BOOZ ALLEN HAMILTON INC. ALL RIGHTS RESERVED.\n\nFOREWORDData Science touches every aspect of our lives on a daily basis. When we visit the doctor, drive our cars, get on an airplane, or shop for services, Data Science is changing the way we interact with and explore  our world.  Our world is now measured, mapped, and recorded in digital bits. Entire lives, from birth to death, are now catalogued in the digital realm. These data, originating from such diverse sources as connected vehicles, underwater microscopic cameras, and photos we post to social media, have propelled us into the greatest age of discovery humanity has ever known. It is through Data Science that we are unlocking the secrets hidden within these data. We are making discoveries that will forever change how we live and interact with the world around us. The impact of these changes is having a profound effect on humanity. We have prope

In [ ]:
offline_pdf_loader = load_document("Rapid Trance - Technical Specification Document (TSD).pdf")
offline_pdf_loader

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


[Document(page_content='Technical Specification Document (TSD)\n\nRapid Trance\n\nVersion 1.0\n\nTable of Contents\n\n1. Document History\n\n2. Technology Stack\n\n3. Assumptions and Risks\n\n4. Repository Naming Convention\n\n5. Git Branch Naming Convention\n\n6. Git Commit Naming Convention\n\n7. Pull Request Guidelines\n\n8. Quality Process and Methodologies\n\n8.1 Security Testing\n\n8.2 QA Deliverables\n\n9. Deployment Guidelines\n\n9.1. Frontend deployment\n\n10. Architecture\n\n10.1. High\n\n\n\nLevel Architecture\n\n10.2. Backend Flask Architecture\n\n10.3. Backend Workflow\n\n10.4. Cloud Infrastructure\n\n11. User Stories and Functional Requirements\n\n11.1. Persona #1: Platform User\n\n12. Models\n\n13. Cases the bot will not join/ FAIL.\n\n14.Reference and Supporting Documents\n\n3\n\n4\n\n5\n\n6\n\n6\n\n6\n\n7\n\n8\n\n8\n\n8\n\n9\n\n9\n\n10\n\n10\n\n10\n\n10\n\n13\n\n11\n\n11\n\n16\n\n17\n\n13\n\n1. Document History\n\nVersion Modification\n\nDate\n\nAuthor\n\nSummary of Ch

In [ ]:
recursive_character_text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
recursive_character_text_splitter

In [ ]:
# Split the document into paragraphs
def split_document(document):
    return recursive_character_text_splitter.split_documents(document)

In [ ]:
online_paragraphs = split_document(online_pdf_loader)
print(online_paragraphs)

In [ ]:
offline_paragraphs = split_document(offline_pdf_loader)
print(offline_paragraphs)

[Document(page_content='Technical Specification Document (TSD)\n\nRapid Trance\n\nVersion 1.0\n\nTable of Contents\n\n1. Document History\n\n2. Technology Stack\n\n3. Assumptions and Risks\n\n4. Repository Naming Convention\n\n5. Git Branch Naming Convention\n\n6. Git Commit Naming Convention\n\n7. Pull Request Guidelines\n\n8. Quality Process and Methodologies\n\n8.1 Security Testing\n\n8.2 QA Deliverables\n\n9. Deployment Guidelines\n\n9.1. Frontend deployment\n\n10. Architecture\n\n10.1. High\n\n\n\nLevel Architecture\n\n10.2. Backend Flask Architecture\n\n10.3. Backend Workflow\n\n10.4. Cloud Infrastructure\n\n11. User Stories and Functional Requirements\n\n11.1. Persona #1: Platform User\n\n12. Models\n\n13. Cases the bot will not join/ FAIL.\n\n14.Reference and Supporting Documents\n\n3\n\n4\n\n5\n\n6\n\n6\n\n6\n\n7\n\n8\n\n8\n\n8\n\n9\n\n9\n\n10\n\n10\n\n10\n\n10\n\n13\n\n11\n\n11\n\n16\n\n17\n\n13\n\n1. Document History\n\nVersion Modification\n\nDate\n\nAuthor\n\nSummary of Ch

In [ ]:
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

In [ ]:
# Connect to the Pinecone vector store
pinecone.init(api_key=os.getenv("PINECONE_API_KEY"),
              environment=os.getenv("PINECONE_ENVIRONMENT"),
              service_name=os.getenv("PINECONE_SERVICE_NAME"))

In [ ]:
# Define a function to create index in pinecone
def create_index(index_name, vector_dim):
    active_indexes = pinecone.list_indexes()
    if index_name not in active_indexes:
        pinecone.create_index(index_name, dimension=vector_dim)
    return pinecone.Index(index_name=index_name)

In [ ]:
index_name = 'rapid-trance-index'
namespace = 'pdf-namespace'


online_doc_search = Pinecone.from_texts(
    [t.page_content for t in online_paragraphs],
    embeddings,
    index_name=index_name,
    namespace=namespace
)
    
print(online_doc_search)

In [ ]:
index_name = 'rapid-trance-index'
namespace = 'pdf-namespace'


offline_doc_search = Pinecone.from_texts(
    [t.page_content for t in offline_paragraphs],
    embeddings,
    index_name=index_name,
    namespace=namespace
)
    
print(offline_doc_search)

In [ ]:
query = "List of all the functional requirement?"

In [ ]:
online_docs = online_doc_search.similarity_search(query, include_metadata=True)
print(online_docs)

In [ ]:
offline_docs = offline_doc_search.similarity_search(query, include_metadata=True)
print(offline_docs)

[Document(page_content='your Data Science project, the skills requirements of the team will change. You will need to re-balance the equation to ensure the reactants balance with  the products.THE FIELD GUIDE to  DATA SCIENCE', metadata={}), Document(page_content='meta-analysis during product reviews.Booz Allen Hamilton was engaged to research, develop, and evaluate emerging informatics tools, methods, and techniques to determine their ability to address regulatory challenges faced by the FDA Center for Drug Evaluation and Research (CDER). The main goal was to enable the CDER community to fully utilize the agency’s expansive data resources for efﬁcient and effective drug review through the design and development of informatics capabilities based on Natural Language Processing (NLP), data integration, and data visualization methodologies.Our ApproachTo support transformational change at CDER, we designed and developed a set of informatics prototypes for the analysis and modeling of compl

In [ ]:
llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)

In [ ]:
# Stuff
chain = load_qa_chain(llm, chain_type="stuff")

In [ ]:
chain.run(input_documents=offline_docs, question=query)

'\n\n1. As an admin User should have the option to select the gas fee depending on the demand for each particular function.\n2. The system should display Liquity Loans protocol statistics (a. ETH, LUSD and LQTY prices with links to Coingecko page for token, price charts, etherscan details and local DEX pair, b. Current Borrowing fee rate, c. TVL for the LL protocol, d. Number of vaults, e. LUSD supply, f. LUSD in stability pool (total $ and % of supply), g. Total collateral ratio (for the protocol), h. Recovery mode status, i. Recovery mode price threshold, j. Kickback rate for the front-end, k. LL contract version, date/time deployed, front-end version).\n3. User Should be able to borrow and repay loans.\n4. User Should be able to create new vault/trove, deposit ETH.\n5. User Should be able to know available loan size (target CR loan value - current debt) - in RED if current CR is under 103% of target CR Green if equal to or greater than 103% of target CR (Green means recommended to c

In [ ]:
# map_reduce
chain = load_qa_chain(llm, chain_type="map_reduce")
chain.run(input_documents=offline_docs, question=query)

' The functional requirements are: \n- ETH, LUSD and LQTY prices with links to Coingecko page for token, price charts, etherscan details and local DEX pair \n- Current Borrowing fee rate \n- TVL for the LL protocol \n- Number of vaults \n- LUSD supply \n- LUSD in stability pool (total $ and % of supply) \n- Total collateral ratio (for the protocol) \n- Recovery mode status \n- Recovery mode price threshold- \n- Kickback rate for the front-end \n- LL contract version, date/time deployed, front-end version \n- User should be able to create new vault/trove, deposit ETH\n- Loan Compounding target selector \n- Top up Vault (LUSD->ETH ->LL Vault)\n- New BOXABL NFT (LUSD->BOXABL NFT market)\n- Top up Stability Pool (LUSD->Stability Pool)\n- Top up Staking Pool (LUSD->LQTY->Staking Pool)\n- Top up staked WATT (LUSD->WATT->CORE Staking)\n- Select up to three places to'

In [ ]:
# refine
chain = load_qa_chain(llm, chain_type="refine")
chain.run(input_documents=offline_docs, question=query)

'\n\n1. Admin users should have the option to select the gas fee depending on the demand for each particular function.\n2. Logic/financial flow schematic should be provided.\n3. Wireframes should be provided.\n4. Figma designs should be provided.\n5. Logo, colors and layout should be provided.\n6. Investor pitch deck should be provided.\n7. Other designs should be provided.\n8. Committed deliverables should be outlined in Appendix A.\n9. Requirements for later phases should be captured in Appendix B.\n10. Important references and similar platforms should be provided.\n11. The system should display Liquity Loans protocol statistics, including ETH, LUSD and LQTY prices with links to Coingecko page for token, price charts, etherscan details and local DEX pair, current borrowing fee rate, TVL for the LL protocol, number of vaults, LUSD supply, LUSD in stability pool (total $ and % of supply), total collateral ratio (for the protocol), recovery mode status, recovery mode price threshold, ki

In [ ]:
# map_rerank
chain = load_qa_chain(llm, chain_type="map_rerank")
chain.run(input_documents=offline_docs, question=query)

'User Should be able to create new vault/trove, deposit ETH'

In [ ]:
query = "which all Technology you are using in the project?"

In [ ]:
offline_docs = offline_doc_search.similarity_search(query, include_metadata=True)
print(offline_docs)

[Document(page_content='Published on September 09, 2022 - use major version for new phases or major changes\n\nMonth DD, YYYY\n\n<<author\n\n\n\nname>>\n\nsummary of changes (can be bulleted)\n\nMonth DD, YYYY\n\n<<author\n\n\n\nname>>\n\nuse minor version for fixes or updates after client reviews\n\n2. Technology Stack\n\nWeb Application\n\nPurpose\n\nCommon\n\nFrontend stack\n\nBackend\n\nDatabase\n\nInfrastructure\n\nPurpose\n\nStorage of Metadata\n\nFrontend\n\nContainerisation\n\nVPC\n\nCI/CD\n\nPython\n\nPurpose\n\nTechnology\n\nPython | Pycharm\n\nReactJS | Redux | Webpack | Styled Components | Formik\n\nPython | Flask | MongoDB\n\nMongoDB\n\nTechnology\n\nAWS S3\n\nAWS Amplify | AngularJS\n\nAWS ECS\n\nPrivate | Public Subnets\n\nAWS ECR | AWS Code Build | AWS Code Pipeline\n\nTechnology\n\nMeetings, Transcripts, User\n\nMajor SDKs/Libraries\n\nGoogle Calendar API\n\nBeautifulsoup | google oauth\n\n| Python Selenium | Python\n\nFlask, pymongo, nltk, boto3,\n\nsummarizer, Openai

In [ ]:
# Stuff
chain = load_qa_chain(llm, chain_type="stuff")

In [ ]:
chain.run(input_documents=offline_docs, question=query)

' The technologies used in the project include Python, Pycharm, ReactJS, Redux, Webpack, Styled Components, Formik, Flask, MongoDB, AWS S3, AWS Amplify, AngularJS, AWS ECS, Private and Public Subnets, AWS ECR, AWS Code Build, AWS Code Pipeline, Google Calendar API, Beautifulsoup, google oauth, Python Selenium, Flask, pymongo, nltk, boto3, summarizer, Openai, sentence transformers, sentry, and Bert extractive.'

In [ ]:
query = "which all Models you are using in the project?"

In [ ]:
offline_docs = offline_doc_search.similarity_search(query, include_metadata=True)
print(offline_docs)

[Document(page_content='Model ValidationRepeating what you just heard does not mean that you  learned anything. Model validation is central to construction of any model. This answers the question “How well did my hypothesis fit the observed data?”If we do not have enough data, our models cannot connect the dots. On the other hand, given too much data the model cannot think outside of the box. The model learns specific details about the training data that do not generalize to the population. This is the problem of model over fitting. Many techniques exist to combat model over fitting. The simplest method is to split your dataset into training, testing and validation sets. The training data is used to construct the model. The model constructed with the training data is then evaluated with the testing data. The performance of the model against the testing set is used to further reduce model error. This indirectly includes the testing data within model construction, helping to reduce model

In [ ]:
# Stuff
chain = load_qa_chain(llm, chain_type="stuff")

In [ ]:
chain.run(input_documents=offline_docs, question=query)

' We are using the following models in the project: bert-base-uncased, action_item_model, MEETING_SUMMARY, emotions, all MiniLM L6 v2 (Sentence Transformer Model), paraphrase MiniLM L6 v2 (SBertSummarizer Model).'